In [ ]:
from nixtla import NixtlaClient

In [ ]:
nixtla_client = NixtlaClient(api_key="nixak-sVs734PNn3JEOMYhbwrfTAW5C9Y1Jqu3jDBlBW38DgOxX8tLmcZSmX4InxwXiiEFXllPOCcGmdKEYSZW")

In [ ]:
#Verifying API key, was able to access this after creating account and following manual at
#https://nixtlaverse.nixtla.io/nixtla/docs/getting-started/quickstart.html#step-1-create-a-timegpt-account-and-generate-your-api-key
nixtla_client.validate_api_key()

In [ ]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt

import cufflinks as cf
from plotly import __version__
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot

cf.go_offline()

%matplotlib inline

In [ ]:
df = pd.read_csv("flu-hospital-admissions.csv", parse_dates = ["date"])
df = df[df["location_name"] == "US"]
df

In [ ]:
df["date"]

In [ ]:
#Traditional plot, works easily with pandas data frame
#Problem can potentially arise when time/date column is the index, which is the standard practice for a normal matplotlib/plotly/pandas plotting flow
nixtla_client.plot(df, time_col="date", target_col="value")

In [ ]:
#No need to sort or set index, but heavy limitations on setting axis title, date ranges, etc.

In [ ]:
#A lot of reformatting of dataset to fit plotly or matplotlib plotting specifications
plotly_df = df[['date','value']]
plotly_df['date'] = pd.to_datetime(plotly_df['date'])
plotly_df.set_index('date', inplace=True)
plotly_df.sort_values(by="date", inplace=True)
plotly_df

In [ ]:
#Reference point to compare visual plots
plotly_df["value"].iplot(yTitle = "Count", xTitle = "Date", title = "US Flu Hospital Admissions")

In [ ]:
#Forecasting which returns data frame. frequency (freq) is a parameter, I've left it blank as the library is supposed to automatically detect frequency
#model is also a parameter, I've left it at default
timegpt_fcst_df = nixtla_client.forecast(df=df, time_col='date', target_col='value', h=12)
timegpt_fcst_df

In [ ]:
#Easy to plot the training df and the forecasted df
nixtla_client.plot(df, timegpt_fcst_df, time_col='date', target_col='value')

In [ ]:
#Little case study. Wanted to see what the prediction would look like in a big spike in December

In [ ]:
#Setting up training data frame to be only US data and exclude last 4 weeks (December)
training_df = df.copy()
training_df = training_df.sort_values(by="date")
training_df = training_df.iloc[:-4]
training_df

In [ ]:
#Forecast next 4 weeks based on this dataframe
forecast_df = nixtla_client.forecast(df=training_df, time_col='date', target_col='value', h=4)
forecast_df = forecast_df.set_index("date")
forecast_df

In [ ]:
#Creating data frame that has one column as training data, one column as real data, and one column as TimeGPT prediction
plotting_df = pd.DataFrame()
plotting_df.index = plotly_df.index
plotting_df["Training Data"] = plotly_df["value"].iloc[:-4]
plotting_df["Real Data"] = plotly_df["value"].iloc[-4:]
plotting_df["TimeGPT Prediction"] = forecast_df["TimeGPT"]
plotting_df

In [ ]:
plotting_df.iplot(xTitle = "Date", yTitle = "Count", title = "TimeGPT vs Real Data")

In [ ]:
timegpt_fcst_df